### testing S2 imagery composites
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

So far 'max' composite from the S2A product is not very good (too much cloud). If want to use 'max' composites, might need to try 95th %ile rather than max, or possibly a more aggressive cloud mask. use of the 'S2A cloudless' product could be a good option for this: https://medium.com/google-earth/more-accurate-and-flexible-cloud-masking-for-sentinel-2-images-766897a9ba5f

In [1]:
import ee
from geemap import geemap

In [12]:
def maskS2clouds(image):
    cloudBitMask = ee.Number(2).pow(10).int()
    cirrusBitMask = ee.Number(2).pow(11).int()
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

# rename bands
#def renameS2bands(image):
#    return image.rename('B', 'G', 'R', 'NIR', 'SWIR', 'SWIR2', 'QA60')
def renameS2bands(image):
    return image.rename('B2', 'B3', 'B4', 'B8', 'B11', 'B12')


rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'], 
}

In [3]:
fp_train_ext1 = "/home/markdj/Dropbox/artio/polesia/val/Vegetation_extent_rough.shp"
fp_roi = "/home/markdj/Dropbox/artio/polesia/val/roi_rough.shp"

In [4]:
roi_test = geemap.shp_to_ee(fp_roi)
roi_train = geemap.shp_to_ee(fp_train_ext1)

In [76]:
start_date_list=['2019-05-01']

### mask cloud using QA60 band

In [28]:
start_date = start_date_list[0]
# load Sentinel-2 TOA reflectance data.
s2collection = ee.ImageCollection('COPERNICUS/S2_SR') \
              .filterDate(start_date, ee.Date(start_date).advance(1, 'month')) \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \
              .filterBounds(roi_train) \
              .select('B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'QA60') \
              .map(maskS2clouds) #\
              #.map(renameS2bands)

#Reduce the collection, and clip to train roi
s2_median = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').median().clip(roi_train.geometry())
s2_min = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').min().clip(roi_train.geometry())
s2_max = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').max().clip(roi_train.geometry())
s2_75 = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').reduce(ee.Reducer.percentile([75])).clip(roi_train.geometry())
#Reducer.percentile has an outputNames arg, but don't think option to give names matching the input names 
s2_75 = renameS2bands(s2_75)

# data stack
#s2_multiple = s2_median.addBands(s2_min).addBands(s2_max)

In [42]:
Map = geemap.Map(center=(51.85, 27.8), zoom=9)
#Map.add_basemap('OpenStreetMap.BlackAndWhite')
#Map.add_basemap('SATELLITE')

Map.addLayer(s2_min, rgbVis, 's2min')
Map.addLayer(s2_median, rgbVis, 's2med')
Map.addLayer(s2_max, rgbVis, 's2max')
Map.addLayer(s2_75, rgbVis, 's2p75')
#Map.addLayer(roi_train, {}, 'ROI_train')
#Map.addLayer(roi_test, {}, 'ROI_test')

Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

Cloud masking in the above is not very good... cloud visible in the median and 75 %ile images. The below compares the existing approach to a slightly different masking function from here: https://courses.spatialthoughts.com/end-to-end-gee.html

In [72]:
# Get the Level-2A Surface Reflectance image
imageSR = ee.Image('COPERNICUS/S2_SR/20190703T050701_20190703T052312_T43PGP')

Map = geemap.Map(center=(51.85, 27.8), zoom=9)
Map.centerObject(imageSR)

# // Function to remove cloud and snow pixels from Sentinel-2 SR image
# function maskCloudAndShadowsSR(image) {
#   var cloudProb = image.select('MSK_CLDPRB');
#   var snowProb = image.select('MSK_SNWPRB');
#   var cloud = cloudProb.lt(5);
#   var snow = snowProb.lt(5);
#   var scl = image.select('SCL'); 
#   var shadow = scl.eq(3); // 3 = cloud shadow
#   var cirrus = scl.eq(10); // 10 = cirrus
#   // Cloud probability less than 5% or cloud shadow classification
#   var mask = (cloud.and(snow)).and(cirrus.neq(1)).and(shadow.neq(1));
#   return image.updateMask(mask);
# }

## Function to remove cloud and snow pixels from Sentinel-2 SR image
## MdJ:i think if you want to use a function with 'map', you can't pass args?
def maskCloudAndShadowsSR(image):
    cloudProb = image.select('MSK_CLDPRB')
    snowProb = image.select('MSK_SNWPRB')
    cloud = cloudProb.lt(2)
    snow = snowProb.lt(2)
    scl = image.select('SCL') 
    shadow = scl.eq(3) # 3 = cloud shadow
    cirrus = scl.eq(10) # 10 = cirrus
    # Cloud probability less than 5% or cloud shadow classification
    #mask = (cloud.and(snow)).and(cirrus.neq(1)).and(shadow.neq(1))
    mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))
    return image.updateMask(mask)

imageSRmasked1 = maskS2clouds(imageSR)
imageSRmasked2 = maskCloudAndShadowsSR(imageSR)

Map.addLayer(imageSR, rgbVis, 'SR Image')
Map.addLayer(imageSRmasked1, rgbVis, 'SR Image masked1')
Map.addLayer(imageSRmasked2, rgbVis, 'SR Image masked2')
Map

Map(center=[12.658204357875261, 76.8410807013455], controls=(WidgetControl(options=['position', 'transparent_b…

new approach looks better. let's try to apply it to the polesia composites...

In [82]:
start_date = start_date_list[0]
# load Sentinel-2 TOA reflectance data.
s2collection = ee.ImageCollection('COPERNICUS/S2_SR') \
              .filterDate(start_date, ee.Date(start_date).advance(3, 'month')) \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \
              .filterBounds(roi_train) \
              .map(maskCloudAndShadowsSR) #\
              #.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'QA60') \
              #.map(renameS2bands)

#Reduce the collection, and clip to train roi
s2_median = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').median().clip(roi_train.geometry())
s2_min = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').min().clip(roi_train.geometry())
s2_max = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').max().clip(roi_train.geometry())
s2_75 = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').reduce(ee.Reducer.percentile([75])).clip(roi_train.geometry())
#Reducer.percentile has an outputNames arg, but don't think option to give names matching the input names 
s2_75 = renameS2bands(s2_75)

# data stack
#s2_multiple = s2_median.addBands(s2_min).addBands(s2_max)

Map = geemap.Map(center=(51.85, 27.8), zoom=9)
#Map.add_basemap('OpenStreetMap.BlackAndWhite')
#Map.add_basemap('SATELLITE')

Map.addLayer(s2_min, rgbVis, 's2min')
Map.addLayer(s2_median, rgbVis, 's2med')
Map.addLayer(s2_max, rgbVis, 's2max')
#Map.addLayer(s2_75, rgbVis, 's2p75')

#Map.addLayer(roi_train, {}, 'ROI_train')
#Map.addLayer(roi_test, {}, 'ROI_test')

Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

while the above is much better than the first attempt, there are still a bunch of cloud edge artifacts in the max and 75%ile outputs even when cloud probability thresh is at it's strictest (<2). thinking about it, it makes sense that unless you have really good masking, you will always have contamination of min/max composits by cloud or cloud shadow. maybe better just using median to avoid these issues?

median still contains some cloud not captured by the current makss, but seems to imporve when we aggregate over multiple months. so maybe doing a 3 month summer and winter composite is the best way forwards

Also - try the cloudless product? https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless